<center style="font-size: 48px; line-height: 2">
<h3>Bringing C performance to Python code</h3>

<br/>
**Jan Škoda**<br/>
<a href="http://www.janskoda.cz">www.janskoda.cz</a> | jan.skoda@quantlane.com
<br/>
<img src="http://quantlane.com/img/logo.png" alt="Quantlane" style="width: 500px;"/>

</center>

## Contents

- How fast is Python?
- Cython
 - ...
- Optimization tips

## Benchmark code

In [13]:
def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n - 1)

In [14]:
%timeit factorial(100)

10000 loops, best of 3: 20.5 µs per loop


## Speed comparison

In [72]:
def identity(n):
    return n

In [73]:
%timeit identity(100)
%timeit 3+5

The slowest run took 13.07 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 96.5 ns per loop
100000000 loops, best of 3: 13.7 ns per loop


| Operation | Duration in Python | Duration in C | Duration in Cython |
|-----------|--------------------|---------------|--------------------|
| Function call | 60 ns | 3 ns | 3 ns | 
| factorial(100) | 20 us | - | 0.1 us | 
| addition | 25 ns | < 1ns | < 1ns |
| longest_common_substring | 142 ms | - | 1.12 ms |

# Cython

> Cython is an *optimising static compiler* for both the Python programming language and the extended Cython programming language.

Features:
- write Python code that calls back and forth from and to C or C++ code natively at any point.
- easily tune readable Python code into plain C performance by adding static type declarations.
- quickly build your applications using distutils/setup.py and distribute them


## Cython factorial

In [2]:
%load_ext Cython

In [3]:
%%cython -a

def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n - 1)

In [39]:
%timeit factorial(100)

100000 loops, best of 3: 15 µs per loop


Note: you can run `cython -a file.pyx` in command line to inspect the 'yellow-analysis' of your code.

In [42]:
%%cython -a

cdef void c_function():
    pass

cpdef int factorial(int n):
    if n == 0:
        return 1
    return n * factorial(n - 1) 

In [43]:
# We can't call c_function() here
%timeit factorial(100)

The slowest run took 9.19 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 183 ns per loop


## String manipulation in Cython

In [4]:
%%cython -a
cimport cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
cpdef str longest_common_substring(str s1, str s2):
    cdef int len_1 = len(s1) + 1
    cdef int len_2 = len(s2) + 1
    cdef:
        np.ndarray[np.int32_t, ndim = 2] m = np.ndarray([len_1, len_2], dtype = np.int32)
        int longest = 0
        int x_longest = 0
        int x, y
    m[0, 0] = 0
    for x in range(1, len_1):
        for y in range(1, len_2):
            if s1[x - 1] == s2[y - 1]:
                m[x, y] = m[x - 1, y - 1] + 1
                if m[x, y] > longest:
                    longest = m[x, y]
                    x_longest = x
            else:
                m[x, y] = 0 
    return s1[x_longest - longest: x_longest]

In [14]:
a = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer tempus vehicula nisl ac pulvinar. Fusce ligula nunc, commodo non tincidunt ac, viverra quis turpis. Vestibulum vitae pellentesque lorem. Aenean a interdum urna, a finibus massa. Maecenas sed vestibulum orci. Nam volutpat mauris nec odio blandit, nec consectetur orci feugiat. Nulla eu libero pharetra, vestibulum magna nec, fermentum elit. Vestibulum sed ipsum ac purus fringilla tristique ut eu risus. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Sed congue ultrices diam a convallis. Vivamus ut tempor nulla.'
b = 'Suspendisse vestibulum eleifend libero sed rhoncus. Interdum et malesuada fames ac ante ipsum primis in faucibus. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Donec aliquam sed eros ac pretium. Duis quis lacus vel nibh vehicula rhoncus et at odio. Curabitur venenatis feugiat nunc. Aenean augue orci, porta sit amet diam a, consequat laoreet lorem. Suspendisse eget felis sagittis nulla finibus bibendum. Nam non diam mi.'
%timeit longest_common_substring(a, b)
longest_common_substring(a, b)

1000 loops, best of 3: 1.14 ms per loop


' ante ipsum primis in faucibus'

## Cython & C++ STL types

In [4]:
%%cython
# distutils: language = c++
from libcpp.string cimport string
from libcpp.vector cimport vector

# iterator -> std::vector -> list
cdef vector[int] vect = range(1, 10, 2)
with nogil:
    vect.push_back(11)
print(vect)

# bytes <-> std::string
cdef vector[string] cpp_strings = b'ab cd ef gh'.split()
print(cpp_strings[1])

[1, 3, 5, 7, 9, 11]
b'cd'


## Parallelism

In [1]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force
# distutils: language = c++
from libcpp.vector cimport vector
from cython.parallel import prange
cimport openmp

cpdef list parallel():
    cdef int i
    cdef vector[int] v
    v.resize(8)
    for i in prange(8, nogil=True):
        v[i] = openmp.omp_get_thread_num()
    return v

ERROR:root:Cell magic `%%cython` not found.


In [5]:
parallel()

[0, 0, 1, 1, 2, 2, 3, 3]

## Cython syntax

- `import`/`cimport`
- `with (no)gil:`
- `cdef type identifier`
- `def`/`cdef`/`cpdef`
- `vector[int]` for template parameters
- `<int>` for type casting

## How to create a Cython module

- pyx -- python code
- pxd -- *header* file for accessing C code
- setup.py

In [ ]:
# setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
    name = "My hello app",
    ext_modules = cythonize('hello.pyx'),
)

### Building

You can build your extensions without generating package using
> python setup.py build_ext --inplace

which will process hello.pyx -> hello.c -> hello.so/hello.pyd. Then you can import the module from python:
> import hello


### Requirements: 
 - Cython
 - gcc

Anaconda python distribution contains a compiler. Pip-wheel can be used to distribute pre-built packages.

## Non-separated python code

In [6]:
%%cython -a
# cython: annotation_typing=True
import cython

# cpdef method using a decorator
@cython.ccall
@cython.returns(int)
@cython.locals(n = int)
def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n - 1)

# cpdef method using type annotations
def add(a: int, b: int) -> int:
    assert cython.compiled
    return a + b

In [49]:
%timeit factorial(100)
add(5, 3)

The slowest run took 11.43 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 146 ns per loop


8

## PyPy

- alternative Python interpreter with JIT compiler
- 3.5 in beta state
- GIL-less branch in alpha state
- supports C extensions, although they are not binary compatable

In [ ]:
%%pypy

def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n - 1)

1000000 loops, best of 1: 4.731 µs per loop

## Optimization tips

- always profile before optimization!
 - `python -m profile -s cumtime program.py`
 - cProfile + snakeviz
 - pytest-benchmark, pytest-profiling
- consider what is worth optimizing and what is not
 - 5x speedup of function taking 10% of runtime means 8.7% speedup overall

- don't forget to write tests before optimizing

# Q&A

<center style="font-size: 48px; line-height: 2">

<br/>
**Jan Škoda**<br/>
<a href="http://www.janskoda.cz">www.janskoda.cz</a> | jan.skoda@quantlane.com
<br/>
<img src="http://quantlane.com/img/logo.png" alt="Quantlane" style="width: 500px;"/>
<br/>
<span style="color:gray; font-size: 32px">sources & references: <a style="color:gray" href="http://github.com/leftys/fast-python">http://github.com/leftys/fast-python</a></span>

</center>

## References
- http://cython.readthedocs.io/en/latest/index.html
- https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/
- https://stackoverflow.com/questions/34189761/distribute-pre-compiled-cython-code-to-windows
- https://cython.readthedocs.io/en/latest/src/reference/compilation.html#compilation-reference

### TODO:

- my contact, quantlane logo and hiring info for the last slide
- use more fragments
- oddelit dva priklady v ## Non-separated python code
- pridat asyncio future example?
- snakeviz screenshot
- okouknout sablonu na talky
- smazat tu cdef funkci a ukazat na dvou slidech nejdriv cdef faktorial, ktery se neda zavolat a pak cpdef verzi
- vysvetlit openmp